### Load the data

In [1]:
import numpy as np
import pandas as pd

table = pd.read_csv("positionData.csv", low_memory=False)
display(table.head())

,TOPchampId,TOPkeyRune,TOPspell1,TOPspell2,JNGchampId,JNGkeyRune,JNGspell1,JNGspell2,MIDchampId,MIDkeyRune,MIDspell1,MIDspell2,ADCchampId,ADCkeyRune,ADCspell1,ADCspell2,SUPchampId,SUPkeyRune,SUPspell1,SUPspell2
0,58,2,4,0,80,15,4,9,131,8,4,7,123,14,4,0,113,14,4,5
1,19,2,4,0,54,4,4,9,136,4,4,5,106,9,4,0,86,14,4,5
2,116,2,4,0,79,1,4,9,51,4,4,0,122,3,4,7,75,7,4,5
3,112,13,4,9,97,4,5,9,82,7,4,7,45,3,4,7,10,4,4,5
4,72,4,4,5,97,4,5,9,60,4,4,5,46,3,4,7,105,7,4,1


### Prepare the data extracting x and y

In [ ]:
import time
import itertools

b = time.time()

train_ratio = 0.8

array = table.values[:len(table)]

i = np.arange(len(table))

#1st position, 2nd game, 3rd parameter, but then transposed
x_cac = np.array([array[np.ix_(i,range(4))], 
                  array[np.ix_(i,range(4, 8))], 
                  array[np.ix_(i,range(8, 12))], 
                  array[np.ix_(i,range(12, 16))], 
                  array[np.ix_(i,range(16, 20))]]).transpose(1, 0, 2)
y_cac = np.array([[1, 0, 0, 0, 0], [0, 1, 0, 0, 0], [0, 0, 1, 0, 0], [0, 0, 0, 1, 0], [0, 0, 0, 0, 1]])

perms = np.array(list(itertools.permutations([0, 1, 2, 3, 4])))

e = np.arange(5)
perm = np.arange(len(perms))

#GO

x_train = np.zeros((int(len(array)*train_ratio)*120, 20))
y_train = np.zeros((int(len(array)*train_ratio)*120, 25))

for i in range(int(len(array)*train_ratio)):
    
    x_train[i*120+perm] = x_cac[i][perms[np.ix_(perm,e)]].reshape(len(perms), 20)
    y_train[i*120+perm] = y_cac[perms[np.ix_(perm,e)]].reshape(len(perms), 25)
    
x_test = np.zeros((int(len(array)*(1-train_ratio))*120, 20))
y_test = np.zeros((int(len(array)*(1-train_ratio))*120, 25))

for i in range(int(len(array)*(1-train_ratio))):
    
    x_test[i*120+perm] = x_cac[i+int(len(array)*train_ratio)][perms[np.ix_(perm,e)]].reshape(len(perms), 20)
    y_test[i*120+perm] = y_cac[perms[np.ix_(perm,e)]].reshape(len(perms), 25)

print("Execution time:", round(time.time()-b, 2), "seconds")

### Shuffle the data and check it is ok

In [3]:
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

x_train, y_train = unison_shuffled_copies(x_train, y_train)
x_test, y_test = unison_shuffled_copies(x_test, y_test)

print(x_train[-1])
print(y_train[-1])

[ 78.   4.   4.   9.  64.   0.   4.   7.   3.  14.   4.   5. 108.  15.
   4.   0.   5.   8.   4.   0.]
[0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0.
 0.]


### Create the loaders and separate training and testing data

In [4]:
import torch
import torch.utils.data

bs = 64

device = torch.device("cuda:0")

train = torch.utils.data.TensorDataset(torch.from_numpy(x_train), torch.from_numpy(y_train))
trainloader = torch.utils.data.DataLoader(train, batch_size=bs, shuffle=True, num_workers=0)

test = torch.utils.data.TensorDataset(torch.from_numpy(x_test), torch.from_numpy(y_test))
testloader = torch.utils.data.DataLoader(test, batch_size=bs, shuffle=True, num_workers=0)

### Define the Neural Network

In [84]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.i = torch.arange(5).view(1, 5).to(device)
        
        self.id = torch.zeros(5, bs, 141).to(device)
        self.rune = torch.zeros(5, bs, 17).to(device)
        self.spells = torch.zeros(5, bs, 10).to(device)
        
        self.dropEmb = nn.Dropout(p=0.).train()
        self.dropLoc = nn.Dropout(p=0.).train()
        self.dropHid = nn.Dropout(p=0.1).train()
        
        self.idC = nn.Linear(141, 60) #50 best
        self.runeC = nn.Linear(17, 10) #9 best
        self.spellsC = nn.Linear(10, 10)#10 best
        
        self.c = nn.Linear(60+10+10, 70) #40 best
        
        self.fc2 = nn.Linear((70)*5, 250) #40, 200 best
        self.fc3 = nn.Linear(250, 25) #200 best
        #self.fc4 = nn.Linear(400, 25)

    def forward(self, x):
        
        self.id.zero_()
        self.rune.zero_()
        self.spells.zero_()
        
        k1 = x[:, self.i*4].long().transpose(0, 2).view(5, bs, 1)
        k2 = x[:, self.i*4+1].long().transpose(0, 2).view(5, bs, 1)
        k3 = x[:, self.i*4+2].long().transpose(0, 2).view(5, bs, 1)
        k4 = x[:, self.i*4+3].long().transpose(0, 2).view(5, bs, 1)
        
        self.id.scatter_(2, k1, 1)
        self.rune.scatter_(2, k2, 1)
        self.spells.scatter_(2, k3, 1)
        self.spells.scatter_(2, k4, 1)
        
        
        
        x = torch.cat((
            self.dropLoc(F.relu(self.c(torch.cat((
                self.dropEmb(F.relu(self.idC(self.id[0]))), 
                self.dropEmb(F.relu(self.runeC(self.rune[0]))), 
                self.dropEmb(F.relu(self.spellsC(self.spells[0])))
                ), dim=1)))),
            self.dropLoc(F.relu(self.c(torch.cat((
                self.dropEmb(F.relu(self.idC(self.id[1]))), 
                self.dropEmb(F.relu(self.runeC(self.rune[1]))), 
                self.dropEmb(F.relu(self.spellsC(self.spells[1])))
            ), dim=1)))),
            self.dropLoc(F.relu(self.c(torch.cat((
                self.dropEmb(F.relu(self.idC(self.id[2]))), 
                self.dropEmb(F.relu(self.runeC(self.rune[2]))), 
                self.dropEmb(F.relu(self.spellsC(self.spells[2])))
            ), dim=1)))),
            self.dropLoc(F.relu(self.c(torch.cat((
                self.dropEmb(F.relu(self.idC(self.id[3]))), 
                self.dropEmb(F.relu(self.runeC(self.rune[3]))), 
                self.dropEmb(F.relu(self.spellsC(self.spells[3])))
            ), dim=1)))),
            self.dropLoc(F.relu(self.c(torch.cat((
                self.dropEmb(F.relu(self.idC(self.id[4]))), 
                self.dropEmb(F.relu(self.runeC(self.rune[4]))), 
                self.dropEmb(F.relu(self.spellsC(self.spells[4])))
            ), dim=1))))
            ), dim=1)
            
        """
        
        x = torch.cat((
            F.relu(self.c(torch.cat((
                F.relu(self.idC(self.id[0])), 
                F.relu(self.runeC(self.rune[0])), 
                F.relu(self.spellsC(self.spells[0]))
                ), dim=1))),
            F.relu(self.c(torch.cat((
                F.relu(self.idC(self.id[1])), 
                F.relu(self.runeC(self.rune[1])), 
                F.relu(self.spellsC(self.spells[1]))
            ), dim=1))),
            F.relu(self.c(torch.cat((
                F.relu(self.idC(self.id[2])), 
                F.relu(self.runeC(self.rune[2])), 
                F.relu(self.spellsC(self.spells[2]))
            ), dim=1))),
            F.relu(self.c(torch.cat((
                F.relu(self.idC(self.id[3])), 
                F.relu(self.runeC(self.rune[3])), 
                F.relu(self.spellsC(self.spells[3]))
            ), dim=1))),
            F.relu(self.c(torch.cat((
                F.relu(self.idC(self.id[4])), 
                F.relu(self.runeC(self.rune[4])), 
                F.relu(self.spellsC(self.spells[4]))
            ), dim=1)))
        ), dim=1)
        
        """
        
        #x = self.dropHid(F.relu(self.fc2(x)))
        x = F.relu(self.fc2(x))
        x = F.sigmoid(self.fc3(x))

        return x

net = Net()
net.to(device)

Net(
  (dropEmb): Dropout(p=0.0)
  (dropLoc): Dropout(p=0.0)
  (dropHid): Dropout(p=0.1)
  (idC): Linear(in_features=141, out_features=60, bias=True)
  (runeC): Linear(in_features=17, out_features=10, bias=True)
  (spellsC): Linear(in_features=10, out_features=10, bias=True)
  (c): Linear(in_features=80, out_features=70, bias=True)
  (fc2): Linear(in_features=350, out_features=250, bias=True)
  (fc3): Linear(in_features=250, out_features=25, bias=True)
)

### Define a loss function and a optimizer

In [85]:
import torch.optim as optim

#optimizer = optim.SGD(net.parameters(), lr=0.2, momentum=0.9)
optimizer = optim.Adam(net.parameters(), amsgrad=True)

### Train the network

In [ ]:
for epoch in range(12):  # loop over the dataset multiple times
    
    testiter = iter(testloader)
    running_loss = 0.0 # mean loss in the 2000 batches between printing and printing
    bTime = time.time()
    
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        
        loss = F.binary_cross_entropy(outputs, labels.float())
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 100 mini-batches
            #print(time.time()-bTime, "iter")
            #bTime = time.time()
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0
            
        if i % 4000 == 3999:
            lossc = 0
            count = 0
            net.train(False)
            for i in range(1000):
                images, labels = testiter.next()
                images, labels = images.to(device), labels.to(device)
                outputs = net(images)
                lossc += F.binary_cross_entropy(outputs, labels.float()).item()
                count += 1
            print('test loss: '+str(round(lossc/count, 4)))
            net.train(True)
            
        if i > len(trainloader)-2*bs:
            break

print('Finished Training')

D:\ANACONDA\lib\site-packages\torch\nn\functional.py:1025: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


[1,   200] loss: 0.034
[1,   400] loss: 0.032
test loss: 0.0476
[1,   600] loss: 0.032
[1,   800] loss: 0.035


### Test the network

In [140]:
lossc = 0
correct = torch.zeros(6).to(device)
cache = torch.zeros(bs).to(device).long()
count = 0

bTime = time.time()

net.train(False)

for data in testloader:
    images, labels = data
    images, labels = images.to(device), labels.to(device)
    outputs = net(images)
    
    lossc += F.binary_cross_entropy(outputs, labels.float()).item()
    
    cache.zero_()
    
    for i in np.arange(0, 25, 5):
        cache += (torch.argmax(outputs[:, i:i+5], 1) == torch.argmax(labels[:, i:i+5], 1)).long()
    
    correct += torch.bincount(cache).float()
        
    if count > 2000:
        break
    count += 1
    
net.train(True)

print(round(time.time()-bTime, 2))

print('loss: '+str(round(lossc/count, 5)))
print('5 correct: '+str(round(correct[5].item()/(count*bs)*100, 2))+'%')
print('4 correct: '+str(round(correct[4].item()/(count*bs)*100, 2))+'%')
print('3 correct: '+str(round(correct[3].item()/(count*bs)*100, 2))+'%')
print('2 correct: '+str(round(correct[2].item()/(count*bs)*100, 2))+'%')
print('1 correct: '+str(round(correct[1].item()/(count*bs)*100, 2))+'%')
print('0 correct: '+str(round(correct[0].item()/(count*bs)*100, 2))+'%')

D:\ANACONDA\lib\site-packages\torch\nn\functional.py:1025: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


20.53
loss: 0.0467
5 correct: 89.56%
4 correct: 3.1%
3 correct: 6.74%
2 correct: 0.62%
1 correct: 0.04%
0 correct: 0.0%


### Save the weights

In [49]:
import pickle

with open('99.71', 'wb') as fp:
    pickle.dump(list(net.parameters()), fp)